# 先从原始数据层面进行分析 
- by 刘道会
- 2020年7月于重庆大学

# 1.载入读取数据

cornell的数据下载后是7个压缩包，如下所示：

In [ ]:
import os
os.chdir('/home/ldh/Documents/github_repository/cornell_dataset/cornell')#这是我的路径

In [ ]:
os.listdir()

解压操作可以手动完成，无需编程，完后得到这些文件（外加一个背景图片）

In [ ]:
data = os.listdir()
data.sort()
print('\n'.join(data))

查看每个文件夹下的内容：

In [ ]:
os.chdir('01')

In [ ]:
os.listdir()

可以看到，文件共有5种，分别是：
- pcd****.txt     :样本对应的点云文件
- pcd****cneg.txt :样本的失败抓取框标注
- pcd****cpos.txt :样本的正确抓取标注
- pcd****r.png    :样本的RGB图像
- pcd****d.tiff   :由点云生成的样本深度图像（数据集中本不包含此文件，自己写程序生成，具体的程序gg-cnn中有给出）

初步的操作计划先不管点云，围绕RGB和Depth图像和抓取框标注展开，先对其进行可视化

In [ ]:
os.chdir('/home/ldh/Documents/github_repository/cornell_dataset')#这是我的路径

In [ ]:
import glob

使用glob模块来解析所有的目标数据路径：

The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. 

glob模块可以检索出所有符合特定路径格式要求（按照unix 命令行规则）的路径名，但是返回的结果顺序是随机的

详见：https://docs.python.org/3/library/glob.html

In [ ]:
help(glob.glob)

In [ ]:
cornell_path = 'cornell'
graspf = glob.glob(os.path.join(cornell_path,'*','pcd*cpos.txt'))
graspf.sort()

In [ ]:
help(os.path.join)#注意此join非彼join，是os.path模块里面自带的构造文件路径的函数

In [ ]:
graspf[0:10]

用这种方式来查找读取文件的好处非常明显，跟cd到指定目录然后listdir相比，这种可以直接返回完整的文件路径，而不需要再进行构造。

之前也看到了，这数据的格式命名都很有规律，所以可以根据这个抓取文件的路径名来构造其他的文件名

In [ ]:
rgbf = [filename.replace('cpos.txt','r.png') for filename in graspf]
depthf = [filename.replace('cpos.txt','d.tiff') for filename in graspf]

In [ ]:
print('\n'.join(rgbf[0:10]))
print('\n'.join(depthf[0:10]))

可视化一下来看看效果

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

for i in range(9):
    img = Image.open(rgbf[i])
    plt.subplot(331+i)
    plt.imshow(img)
plt.show()

for i in range(9):
    img = Image.open(depthf[i])
    plt.subplot(331+i)
    plt.imshow(img)
plt.show()

图片正常显示，说明文件路径无误。

# 2.对数据进行处理
## 2.1可视化抓取框标注

In [ ]:
#先打开一个文件看一下怎么存的
with open(graspf[0],'r') as f:
    grasp_data = f.read()

In [ ]:
print(grasp_data)

我记得之前的帮助文件里面写了，这个里面就是点的坐标，每一行就是一个点，每四个点就是一个抓取矩形。

因为它按行区分，所以换个读取函数，按行读取比较好

In [ ]:
with open(graspf[0],'r') as f:
    grasp_data = f.readlines()

In [ ]:
grasp_data

In [ ]:
grasp_data = [grasp.strip() for grasp in grasp_data]#去除末尾换行符

In [ ]:
grasp_data

接下来要从里面把框的数据读出来并按照一个框一个框的格式区分

In [ ]:

def str2num(point):
    '''
    :参数  :point,字符串，以字符串形式存储的一个点的坐标
    :返回值 :列表，包含int型抓取点数据的列表[x,y]
    
    '''
    x,y = point.split()
    x,y = int(round(float(x))),int(round(float(y)))
    
    return (x,y)

def get_rectangle(cornell_grasp_file):
    '''
    :参数  :cornell_grap_file:字符串，指向某个抓取文件的路径
    :返回值 :列表，包含各个抓取矩形数据的列表
    
    '''
    grasp_rectangles = []
    with open(cornell_grasp_file,'r') as f:
        while True:
            grasp_rectangle = []
            point0 = f.readline().strip()
            if not point0:
                break
            point1,point2,point3 = f.readline().strip(),f.readline().strip(),f.readline().strip()
            grasp_rectangle = [str2num(point0),
                               str2num(point1),
                               str2num(point2),
                               str2num(point3)]
            grasp_rectangles.append(grasp_rectangle)
    
    return grasp_rectangles

In [ ]:
grs = get_rectangle(graspf[0])

In [ ]:
grs

因为matplotlib只能画这种竖直的矩形：
:                +------------------+
:                |             |
:              height            |
:                |             |
:               (xy)---- width -----+

In [ ]:
import cv2

In [ ]:
img = cv2.imread(rgbf[0])

In [ ]:
for gr in grs:
    for i in range(3):
        img = cv2.line(img,gr[i],gr[i+1],5)
    img = cv2.line(img,gr[3],gr[0],5) 

In [ ]:
cv2.imshow('img',img)
cv2.waitKey(10000)

In [ ]:
plt.imshow(img)
plt.show()

## 完整程序如下：


In [ ]:
import os
import glob
import cv2
import random
import numpy as np

cornell_path = "cornell"
graspf = glob.glob(os.path.join(cornell_path,'*','pcd*cpos.txt'))
graspf.sort()

rgbf = [filename.replace('cpos.txt','r.png') for filename in graspf]
depthf = [filename.replace('cpos.txt','d.tiff') for filename in graspf]


#总结一下部分代码的定位，其实有关后面内容的到这基本就结束了，就是一个读入，后面的几个函数都是可视化抓取框用的，要说用处也有一些，但对训练没用，对人有用而已。



def str2num(point):
    '''
    :功能  :将字符串类型存储的抓取框脚点坐标取整并以元组形式返回
    
    :参数  :point,字符串，以字符串形式存储的一个点的坐标
    :返回值 :列表，包含int型抓取点数据的列表[x,y]
    '''
    x,y = point.split()
    x,y = int(round(float(x))),int(round(float(y)))
    
    return (x,y)#如果想在后面可视化框的话，这里就得返回元组类型的数据，或者后面再类型转换为元组

def get_rectangles(cornell_grasp_file):
    '''
    :功能  :从抓取文件中提取抓取框的坐标信息
    
    :参数  :cornell_grap_file:字符串，指向某个抓取文件的路径
    :返回值 :列表，包含各个抓取矩形数据的列表
    '''
    grasp_rectangles = []
    with open(cornell_grasp_file,'r') as f:
        while True:
            grasp_rectangle = []
            point0 = f.readline().strip()
            if not point0:
                break
            point1,point2,point3 = f.readline().strip(),f.readline().strip(),f.readline().strip()
            grasp_rectangle = [str2num(point0),
                               str2num(point1),
                               str2num(point2),
                               str2num(point3)]
            grasp_rectangles.append(grasp_rectangle)
    
    return grasp_rectangles

def draw_rectangles(img_path,grasp_path):
    '''
    :功能  :在指定的图片上绘制添加相应的抓取标注框
    
    :参数  :img_path:字符串，指向某个RGB图片的路径
    :参数  :grasp_path:字符串，指向某个抓取文件的路径
    :返回值 :numpy数组，已经添加完抓取框的img数组
    '''
    img_path = img_path
    grasp_path = grasp_path
    
    img = cv2.imread(img_path)
    grs = get_rectangles(grasp_path)
    
    for gr in grs:
        #产生随机颜色
        color = (random.randint(0,255),random.randint(0,255),random.randint(0,255))
        #绘制添加矩形框
        for i in range(3): #因为一个框只有四条线，所以这里是3
            img = cv2.line(img,gr[i],gr[i+1],color,3)
        img = cv2.line(img,gr[3],gr[0],color,2) #补上最后一条封闭的线
    
    cv2.imshow('img',img)
    cv2.waitKey(1000)
    
    return img

if __name__ == "__main__":
    img = draw_rectangles(rgbf[0],graspf[0])
#jupyter好像用不了 parser，所以这里就没用